<a href="https://colab.research.google.com/github/ChristianLG2/Other-Projects---Experience/blob/main/BYU_I_R%26A_Data_Checks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import requests
import zipfile
import json
import io, os
import sys
import re
import pandas as pd
import glob
from google.colab import files

def get_env_vars():
    try:
        dataCenter = input("Enter your DATA_CENTER: ")
        apiToken = input("Enter your API_TOKEN: ")
    except:
        print("You must set DATA_CENTER and API_TOKEN")
        sys.exit(2)
    return dataCenter, apiToken


def exportSurvey(apiToken, surveyId, dataCenter, fileFormat):

    # Setting static parameters
    requestCheckProgress = 0.0
    progressStatus = "inProgress"
    baseUrl = "https://{0}.qualtrics.com/API/v3/surveys/{1}/export-responses/".format(dataCenter, surveyId)
    headers = {
        "content-type": "application/json",
        "x-api-token": apiToken,
    }

    # Step 1: Creating Data Export
    downloadRequestUrl = baseUrl
    downloadRequestPayload = '{"format":"' + fileFormat + '"}'
    downloadRequestResponse = requests.request("POST", downloadRequestUrl, data=downloadRequestPayload, headers=headers)
    progressId = downloadRequestResponse.json()["result"]["progressId"]
    print(downloadRequestResponse.text)

    # Step 2: Checking on Data Export Progress and waiting until export is ready
    while progressStatus != "complete" and progressStatus != "failed":
        print ("progressStatus=", progressStatus)
        requestCheckUrl = baseUrl + progressId
        requestCheckResponse = requests.request("GET", requestCheckUrl, headers=headers)
        requestCheckProgress = requestCheckResponse.json()["result"]["percentComplete"]
        print("Download is " + str(requestCheckProgress) + " complete")
        progressStatus = requestCheckResponse.json()["result"]["status"]

    # Step 2.1: Check for error
    if progressStatus == "failed":
        raise Exception("export failed")

    if progressStatus != "complete":
        raise Exception("export did not complete in time")

    fileId = requestCheckResponse.json()["result"]["fileId"]

    # Step 3: Downloading file
    requestDownloadUrl = baseUrl + fileId + '/file'
    requestDownload = requests.request("GET", requestDownloadUrl, headers=headers, stream=True)

    if requestDownload.status_code != 200:
        raise Exception("Failed to download file: status code " + str(requestDownload.status_code))

    # Step 4: Saving the file
    zip_file_path = "/content/MyQualtricsDownload/{}.zip".format(surveyId)
    os.makedirs(os.path.dirname(zip_file_path), exist_ok=True)
    with open(zip_file_path, 'wb') as f:
        f.write(requestDownload.content)

    # Step 5: Unzipping the file
    extract_folder = "/content/MyQualtricsDownload/{}".format(surveyId)
    os.makedirs(extract_folder, exist_ok=True)
    zipfile.ZipFile(zip_file_path).extractall(extract_folder)

    # Assuming there's only one .csv file in the .zip file, find it
    csv_file_path = glob.glob("/content/MyQualtricsDownload/{}/*.csv".format(surveyId))[0]

    return csv_file_path  # Return the path of the downloaded csv file

def load_survey_to_dataframe(file_path):
    # Load the survey results into a DataFrame
    df = pd.read_csv(file_path)
    return df

def main(apiToken, dataCenter):
    survey_id = input("Enter the Survey ID (e.g. SV_....): ")
    surveyId = survey_id

    fileFormat = "csv"

    csv_file_path = exportSurvey(apiToken, surveyId, dataCenter, fileFormat)
    df = load_survey_to_dataframe(csv_file_path)
    print(df.head())  # print the first 5 rows of the DataFrame

    return df  # Return the DataFrame


if __name__== "__main__":
    dataCenter, apiToken = get_env_vars()
    df = main(apiToken, dataCenter)

# Filter the DataFrame to keep only the rows where "RecordedDate" is

# Specify the columns to keep
columns_to_keep = ['RecordedDate', 'SectionID']

# Drop all other columns
df = df.drop(columns=df.columns.difference(columns_to_keep))

#Drop rows
df = df.drop([0,1,2])

# Convert 'RecordedDate' column to datetime type
df['RecordedDate'] = pd.to_datetime(df['RecordedDate'])

start = input("Insert the first (oldest) date (format: YYYY-MM-DD): ")
end = input("Insert the most recent (last) date (format: YYYY-MM-DD): ")

start_date = pd.to_datetime(start).date()
end_date = pd.to_datetime(end).date()

# Filter by the date range
df = df[df['RecordedDate'].dt.date.between(start_date, end_date)]

# Extract course information and create a new BYUISection column
df['BYUISections'] = df['SectionID'].str.extract(r'(\b[A-Z\d\s]+\.[A-Z\d]+\b)')

# Count the number of unique values in the 'BYUISections' column
unique_count = df['BYUISections'].nunique()

# Print the number of unique values
print("Number of unique BYUISections:", unique_count)

# Extract Ensign information and create another new column
df['EnsignSections'] = df.loc[df['SectionID'].str.contains('LDSBC'), 'SectionID'].str.extract(r'LDSBC-(\w+)-(\w+)-(\d+)-(\w+)').apply(lambda x: f"{x[0]} {x[1]}.{x[3]}", axis=1)

df['SectionsCombined'] = df['BYUISections'].fillna('') + df['EnsignSections'].fillna('')

# Specify the columns to keep
columns_to_keep = ['RecordedDate','SectionsCombined']

# Drop all other columns
df = df.drop(columns=df.columns.difference(columns_to_keep))

#Drop duplicates
df = df.drop_duplicates(subset=['SectionsCombined'])

# Import SOT CSV file into a DataFrame
uploaded = files.upload()
sot_file_name = list(uploaded.keys())[0]
sot_df = pd.read_csv(io.StringIO(uploaded[sot_file_name].decode('utf-8')))

# Check if values in df exist in sot_df
df['In_SOT'] = df['SectionsCombined'].isin(sot_df['SectionsCombined'])

# Existing check: are survey sections in SOT?
df['In_SOT'] = df['SectionsCombined'].isin(sot_df['SectionsCombined'])

# New check: are SOT sections in survey responses?
sot_df['In_Survey'] = sot_df['SectionsCombined'].isin(df['SectionsCombined'])

# Now you can print whether all sections from SOT appear in the survey:
print("All sections from SOT are in the survey:", sot_df['In_Survey'].all())


# Create a new DataFrame that only contains the rows where 'In_SOT' is False
new_st = df[df['In_SOT'] == False]

# Save false_df as a new CSV file
new_st.to_csv('/content/path_to_output_file.csv', index=False)


Enter your DATA_CENTER: sjc1
Enter your API_TOKEN: HeRCYlVgUeAv7hsPVsDlkiGJCPwH2BW3XuZF1ouf
Enter the Survey ID (e.g. SV_....): SV_cCHk9tUXPGiDrxz
{"result":{"progressId":"ES_8p4snVDvJ7qphu6","percentComplete":0.0,"status":"inProgress"},"meta":{"requestId":"c12ca964-c0e2-4590-85b5-60174d7c5a92","httpStatus":"200 - OK"}}
progressStatus= inProgress
Download is 0.0 complete
progressStatus= inProgress
Download is 0.0 complete
progressStatus= inProgress
Download is 0.0 complete
progressStatus= inProgress
Download is 0.0 complete
progressStatus= inProgress
Download is 0.0 complete
progressStatus= inProgress
Download is 0.0 complete
progressStatus= inProgress
Download is 0.0 complete
progressStatus= inProgress
Download is 0.0 complete
progressStatus= inProgress
Download is 0.0 complete
progressStatus= inProgress
Download is 0.0 complete
progressStatus= inProgress
Download is 0.0 complete
progressStatus= inProgress
Download is 0.0 complete
progressStatus= inProgress
Download is 0.0 complete
pr

<ipython-input-12-e637d4767986>:82: DtypeWarning: Columns (2,4,5,6,9,10,11,12,13,14,16,17,18,19,20,21,22,24,25,26,27,28,29,30,31,32,33,34,35,36,38,39,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


                                 StartDate  \
0                               Start Date   
1  {"ImportId":"startDate","timeZone":"Z"}   
2                      2016-06-27 06:17:31   
3                      2016-06-27 14:01:02   
4                      2016-06-27 14:04:09   

                                 EndDate                 Status  \
0                               End Date          Response Type   
1  {"ImportId":"endDate","timeZone":"Z"}  {"ImportId":"status"}   
2                    2016-06-27 06:28:58                      0   
3                    2016-06-27 14:02:25                      0   
4                    2016-06-27 14:05:10                      0   

                  IPAddress                 Progress    Duration (in seconds)  \
0                IP Address                 Progress    Duration (in seconds)   
1  {"ImportId":"ipAddress"}  {"ImportId":"progress"}  {"ImportId":"duration"}   
2             67.174.220.15                      100                      687

<ipython-input-12-e637d4767986>:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['BYUISections'] = df['SectionID'].str.extract(r'(\b[A-Z\d\s]+\.[A-Z\d]+\b)')
<ipython-input-12-e637d4767986>:135: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['EnsignSections'] = df.loc[df['SectionID'].str.contains('LDSBC'), 'SectionID'].str.extract(r'LDSBC-(\w+)-(\w+)-(\d+)-(\w+)').apply(lambda x: f"{x[0]} {x[1]}.{x[3]}", axis=1)
<ipython-input-12-e637d4767986>:137: SettingWithCopyWarning: 
A value is trying to be set

Saving data.csv to data (1).csv
All sections from SOT are in the survey: False
